In [1]:
import pandas as pd
import mysql.connector
import numpy as np
from sklearn import tree

def import_data():
    con  = mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="000012300"
        )
    cur = con.cursor()
    cur.execute("USE mini_data")
    cur.execute("SELECT code from symptoms ORDER BY code")
    ##variables
    diseaseCodes = []
    symptomCode = []
    relations=[]

    for i in cur:
        symptomCode.append(i[0])

    cur.execute("SELECT code from diseases ORDER BY code")
    for i in cur:
        diseaseCodes.append(i[0])

    relations =  np.zeros((len(diseaseCodes), len(symptomCode)))

    for i in diseaseCodes:
        cur.execute("SELECT symptom_code from disease_symptom WHERE disease_code= '"+i+" '")
        for k in cur:
            relations[diseaseCodes.index(i)][symptomCode.index(k[0])]=1
        
    # for i in range(len(diseaseCodes)):
    #     print(diseaseCodes[i],end="\t")
    #     for j in range(len(symptomCode)):
    #         print(relations[i][j],end =" ")
    return diseaseCodes,symptomCode,relations

In [2]:
from sklearn.tree.export import export_text
diseaseCodes,symptomCode,relations = import_data()
clf = tree.DecisionTreeClassifier(criterion='entropy', splitter='best')
clf = clf.fit(relations,diseaseCodes)

print(clf.predict([relations[10]]))
print(diseaseCodes[10])

etx = export_text(clf,feature_names=symptomCode)
#print(etx)

['C0004096']
C0004096


/home/rishabh/anaconda3/envs/mini/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.tree.export module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
tree = clf.tree_
node = 0      #Index of root node
while True:
    feat,thres = tree.feature[node],tree.threshold[node]
    print(feat,thres)
    v = float(input("Do you have {}".format(symptomCode[feat])))
    if v<=thres:
        node = tree.children_left[node]
    else:
        node = tree.children_right[node]
    if tree.children_left[node] == tree.children_right[node]: #Check for leaf
        label = np.argmax(tree.value[node])
        print("We've reached a leaf")
        print("Predicted Label is: {}".format(label))
        break

266 0.5


Do you have C0392680 1


30 0.5


Do you have C0013404 0


83 0.5


...

136 0.5


Do you have C0151706 0


129 0.5


Do you have C0086439 0


We've reached a leaf
Predicted Label is: 60


In [6]:
print(diseaseCodes[60])

C0020538


In [33]:
#utility function to diaplay decision tree
def plot_tree(clf,disease,symptoms):
    from sklearn.externals.six import StringIO  
    from IPython.display import Image  
    from sklearn.tree import export_graphviz
    import pydotplus
    dot_data = StringIO()
    export_graphviz(clf, out_file=dot_data,  
                    filled=True, rounded=True,
                    special_characters=True, feature_names = symptoms)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    graph.write_png('diabetes.png')
    Image(graph.create_png())

In [6]:
print(clf.get_depth())

12


In [37]:
plot_tree(clf,diseaseCodes,symptomCode)

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.834084 to fit

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.834084 to fit



In [11]:
import numpy as np
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print ("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print ("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print ("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print ("{}return {}".format(indent, tree_.value[node]))
    recurse(0, 1)

In [12]:
tree_to_code(clf,symptomCode)

def tree(C0000727, C0000731, C0000737, C0002416, C0002962, C0003123, C0003126, C0003862, C0003962, C0004093, C0004134, C0004604, C0006157, C0006318, C0006625, C0007398, C0007859, C0008031, C0008033, C0008301, C0008767, C0009024, C0009806, C0010200, C0010520, C0011991, C0012833, C0013132, C0013144, C0013362, C0013404, C0013428, C0013491, C0014394, C0015672, C0015967, C0016204, C0016382, C0016512, C0016579, C0016927, C0018681, C0018800, C0018834, C0018862, C0018932, C0018965, C0018991, C0019079, C0019080, C0019209, C0019214, C0019572, C0019825, C0020175, C0020303, C0020440, C0020458, C0020461, C0020578, C0020580, C0020598, C0020621, C0020625, C0020639, C0020649, C0020672, C0022107, C0023222, C0023380, C0024031, C0024103, C0026827, C0026961, C0027066, C0027497, C0027498, C0027769, C0028081, C0028084, C0028643, C0028961, C0029053, C0030193, C0030232, C0030252, C0030318, C0030552, C0030554, C0032617, C0032781, C0033774, C0034079, C0034642, C0034880, C0035508, C0036396, C0036572, C0037383, C

In [25]:
#to show the symptom codes for any disease 
con  = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="000012300"
    )
cur = con.cursor()
cur.execute("USE mini_data")
querry = "SELECT symptom_code from disease_symptom WHERE disease_code = '" +diseaseCodes[0]+"'"
print(querry)
cur.execute(querry)
for i in cur:
    print (i)


SELECT symptom_code from disease_symptom WHERE disease_code = 'C0001175'
('C0015967',)
('C0028081',)
('C0233308',)
('C0010200',)
('C0043096',)
('C0085593',)
('C0011991',)
('C0008033',)
('C0376405',)
('C0231835',)
('C0239134',)
('C0026827',)
('C0424000',)
